In [1]:
import os
import ssl
import httpx
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

ssl_context = ssl.create_default_context()
ssl_context.check_hostname = False
ssl_context.verify_mode = ssl.CERT_NONE
http_client = httpx.Client(verify=False)
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY2"), http_client=http_client)

In [2]:
assistant =client.beta.assistants.create(
    name = "Tutor de Tecnologia",
    instructions = "Você é um tutor sobra assuntos relacionados a tecnologia",
    tools = [{"type": "code_interpreter"}],
    model = "gpt-3.5-turbo-0125"
)

In [3]:
pergunta = """
 Em um sistema de cache com mapeamento direto, dado uma memória principal de uma cache
 de 256 KB e blocos de 64 bytes:
 - Quantas linhas existem na cache?
 - Onde o bloco de endereço 0x1A23C será mapeado na cache?
"""

In [4]:
thread = client.beta.threads.create()
message = client.beta.threads.messages.create(
    thread_id = thread.id,
    role = "user",
    content = pergunta
)

C:\Users\fmtorres\AppData\Local\Temp\ipykernel_20236\3685932091.py:1: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  thread = client.beta.threads.create()
C:\Users\fmtorres\AppData\Local\Temp\ipykernel_20236\3685932091.py:2: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  message = client.beta.threads.messages.create(


In [5]:
run = client.beta.threads.runs.create(
    thread_id = thread.id,
    assistant_id = assistant.id,
    instructions = "Nome de usuário premium"
)

C:\Users\fmtorres\AppData\Local\Temp\ipykernel_20236\3264856326.py:1: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run = client.beta.threads.runs.create(


In [8]:
run.status

'completed'

In [7]:
import time
while run.status in ["queued", "in_progress","cancelling"]:
    print(f"Status: {run.status}")
    time.sleep(1)
    run = client.beta.threads.runs.retrieve(
        thread_id = thread.id,
        run_id = run.id
    )

Status: queued


C:\Users\fmtorres\AppData\Local\Temp\ipykernel_20236\3618006381.py:5: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run = client.beta.threads.runs.retrieve(


Status: in_progress
Status: in_progress
Status: in_progress
Status: in_progress


In [9]:
if run.status == "completed":
    mensagens = client.beta.threads.messages.list(
        thread_id = thread.id
    )
    print(mensagens)
else:
    print(f"Erro: {run.status}")


C:\Users\fmtorres\AppData\Local\Temp\ipykernel_20236\687290153.py:2: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  mensagens = client.beta.threads.messages.list(


SyncCursorPage[Message](data=[Message(id='msg_LvI6fU8Yed7lJN1ArkrGEPAI', assistant_id='asst_qDi1bHlusBUwINzhK6ZXOato', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Os bits de índice para mapeamento do bloco de endereço 0x1A23C na cache são: 0001000111100.\n\nEsses bits serão usados para determinar a linha na qual o bloco será mapeado na cache.\n\nSe precisar de mais alguma informação ou cálculo, por favor, me avise!'), type='text')], created_at=1767708664, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_etMP1CTlOzU0UlzDzLgYvzXC', status=None, thread_id='thread_TfvirlOnRz0iWIfvwlb4JABC'), Message(id='msg_u28ETiMQpLGX89zVKetdVl6j', assistant_id='asst_qDi1bHlusBUwINzhK6ZXOato', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='O endereço convertido para binário possui 17 bits. Como a cache possui um total de 256 linhas (considerando 

In [10]:
print(mensagens.data[0].content[0].text.value)

Os bits de índice para mapeamento do bloco de endereço 0x1A23C na cache são: 0001000111100.

Esses bits serão usados para determinar a linha na qual o bloco será mapeado na cache.

Se precisar de mais alguma informação ou cálculo, por favor, me avise!


In [11]:
run_steps = client.beta.threads.runs.steps.list(
    thread_id = thread.id,
    run_id = run.id
)


C:\Users\fmtorres\AppData\Local\Temp\ipykernel_20236\320213380.py:1: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run_steps = client.beta.threads.runs.steps.list(


In [12]:
for step in run_steps.data[::-1]:
    print(f"\n====Step {step.step_details.type}")
    if step.step_details.type == "tool_calls":
        for tool_call in step.step_details.tool_calls:
            print("=" *10)
            print(tool_call.code_interpreter.input)
            print("="*10)
    if step.step_details.type == "message_creation":
        message = client.beta.threads.messages.retrieve(
            thread_id=thread.id,
            message_id=step.step_details.message_creation.message_id
        )
        
        print(message.content[0].text.value)


====Step message_creation


C:\Users\fmtorres\AppData\Local\Temp\ipykernel_20236\1740727927.py:9: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  message = client.beta.threads.messages.retrieve(


Para calcular o número de linhas na cache, primeiro precisamos determinar o tamanho de cada bloco em termos de bits. Com blocos de 64 bytes, podemos calcular o tamanho em bits da seguinte maneira:
- 1 byte = 8 bits
- 64 bytes = 64 * 8 bits

Em seguida, podemos calcular o número de linhas na cache, já que cada bloco tem um mapeamento direto para uma linha da cache.

Para determinar onde o bloco de endereço 0x1A23C será mapeado na cache, precisamos primeiro converter o endereço hexadecimal para binário e, em seguida, identificar os bits relevantes para o mapeamento.

Vamos realizar os cálculos agora.

====Step tool_calls
# Tamanho de cada bloco em bits
tamanho_bloco_bytes = 64
tamanho_bloco_bits = tamanho_bloco_bytes * 8

# Tamanho da cache em KB
tamanho_cache_kb = 256
tamanho_cache_bytes = tamanho_cache_kb * 1024

# Número de linhas na cache (cada bloco mapeia diretamente para uma linha)
num_linhas_cache = tamanho_cache_bytes / tamanho_bloco_bytes

# Endereço em hexadecimal a ser mapead